In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import model_selection
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
test=pd.io.parsers.read_csv("test.csv")

/home/rohit_mattah/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train=pd.io.parsers.read_csv("train.csv", nrows = 20000000)

In [ ]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [ ]:
train1 = train.drop(['ConversionDate','ConversionPayOut','ConversionStatus','ID', 'Browser', 'Device', 'ClickDate','UserIp','Fraud', 'OS', 'TrafficType'], axis=1)

In [ ]:
#train1 = pd.get_dummies(data=train1, columns=['OS','TrafficType'])
train1['RefererUrl'] = number.fit_transform(train1['RefererUrl'].astype('str'))
train1['Country'] = number.fit_transform(train1['Country'].astype('str'))
train1['subPublisherId'] = number.fit_transform(train1['subPublisherId'].astype('str'))
train1['publisherId'] = number.fit_transform(train1['publisherId'].astype('str'))
x = train1.values

In [ ]:
y=train['ConversionPayOut'].ravel()

In [ ]:
t1 = test.drop(['ID', 'Browser', 'Device', 'ClickDate','UserIp','Fraud','OS', 'TrafficType'], axis=1)
#t1 = pd.get_dummies(data=t1, columns=['OS','TrafficType'])
t1['RefererUrl'] = number.fit_transform(t1['RefererUrl'].astype('str'))
t1['Country'] = number.fit_transform(t1['Country'].astype('str'))
t1['subPublisherId'] = number.fit_transform(t1['subPublisherId'].astype('str'))
t1['publisherId'] = number.fit_transform(t1['publisherId'].astype('str'))
testA = t1.values

In [ ]:
t1.to_csv("testfinal.csv", index=False)

In [ ]:
testfinal=pd.io.parsers.read_csv("testfinal.csv")

In [ ]:
testA = testfinal.values

In [ ]:
xgb1 = xgb.XGBRegressor(max_depth=2, learning_rate=0.01, n_estimators=500, silent=True, objective='reg:linear',
                         min_child_weight=1, subsample=0.5, colsample_bytree=0.5, gamma = 4, 
                          scale_pos_weight=0.5, 
                          seed=1000, missing=None)

In [ ]:
model1 = xgb1.fit(x,y)

In [ ]:
import pickle 

In [ ]:
pickle.dump(model1, open("pima.pickle.dat", "wb"))

In [ ]:
loaded_model = pickle.load(open("pima.pickle.dat", "rb"))

In [ ]:
train1.dtypes

In [ ]:
t1.dtypes

In [ ]:
predictsA = loaded_model.predict(testA)

In [ ]:
i1 = pd.DataFrame({ 'ConversionPayOut': predictsA })

In [ ]:
i1.to_csv("pred.csv", index=False)

In [4]:
ty = test['ID']

In [6]:
i1 = pd.io.parsers.read_csv("pred.csv")

In [9]:
i2 = pd.DataFrame({ 'ID': ty })

In [13]:
result = pd.concat([i2, i1], axis=1)

In [14]:
result.head()

,ID,ConversionPayOut
0,63367289,0.028647
1,63367290,0.058445
2,63367291,0.056083
3,63367292,0.010710
4,63367293,0.044805


In [15]:
result.to_csv("result.csv", index=False)

In [ ]:
result = pd.concat([ty,i1],axis=1)

In [ ]:
rf = RandomForestRegressor(n_estimators=5)
svr = SVR(kernel='linear')
lasso = Lasso(alpha=2.0, fit_intercept=True, normalize=True,max_iter=500)
mlp = MLPRegressor(hidden_layer_sizes=(17,15,10), activation='relu',alpha=0.0001,  
                   max_iter=500, shuffle=True, random_state=100)
gb = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05)
stack = StackingCVRegressor(regressors=(gb,lasso, rf, mlp),
                            meta_regressor=xgb)

In [ ]:
#st = model_selection.cross_val_score(svr, x, y, 
                                              cv=5,scoring='neg_mean_squared_error',n_jobs = -1) #Stacking model CV calculation
#st.mean()

In [ ]:
sta = model_selection.cross_val_score(xgb1, x, y, 
                                              cv=2,scoring='neg_mean_squared_error',n_jobs = -1) #Stacking model CV calculation
sta.mean()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
xgb.plot_importance(model1)

In [ ]:
import gc
gc.collect()

In [ ]:
del testfinal